In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 使 GPU 0 和 1 可见

import sys
sys.path.append("..")
sys.path.append("../mango")
sys.path.append("../mango/design_baselines/diff")


import numpy as np
import torch
import pytorch_lightning as pl

import mango_jupyter_scripts
import design_bench
from helpers import seed_everything 

from SDE import DiffusionSOO



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
#!/usr/bin/env python3
"""
Main driver program for the mango as one of baseline methods for design-bench.

"""
import argparse
import logging
def build_args() -> argparse.Namespace:
    """
    Defines the experimental arguments for offline MBO baseline method
    evaluation.
    Input:
        None.
    Returns:
        A namespace containing the experimental argument values.
    """
    parser = argparse.ArgumentParser(description="Our method Experiments")

    parser.add_argument(
        "--task",
        type=str,
        default='DKittyMorphology-Exact-v0', 
        choices=[task.task_name for task in design_bench.registry.all()],
        help="The name of the design-bench task for the experiment."
    )
    parser.add_argument(
        "--logging-dir",
        type=str,
        default=None,
        help="Logging directory to save optimization results to."
    )
    parser.add_argument(
        "--budget", type=int, default=2048, help="Query budget. Default 2048."
    )
    parser.add_argument(
        "--hidden-size",
        type=int,
        default=2048,
        help="Hidden size of the surrogate objective model. Default 2048."
    )
    parser.add_argument(
        "--seed", type=int, default=2024, help="Random seed. Default 42."
    )
    parser.add_argument(
        "--device", type=str, default="auto", help="Device. Default `auto`."
    )
    args = parser.parse_args([])
    return args

args = build_args()

In [ ]:
from mango_jupyter_scripts.training import train_surrogate, mango_train
# Training a surrogate forward model (It is merely to provide a forward model for comparison purposes only).
forwardmodel = train_surrogate(task_name=args.task, hidden_size=args.hidden_size)

Loading model from ../checkpoints/surrogate/DKittyMorphology-Exact-v0-surrogate-100epoch.ckpt


In [6]:
task = design_bench.make( args.task)
normalize_x = True
normalize_y = True
x_min, x_max, y_min, y_max = np.min(task.dataset.x, axis=0), np.max(task.dataset.x, axis=0), np.min(task.dataset.y, axis=0), np.max(task.dataset.y, axis=0) 
try:
    x1_range = task.dataset.x1_range
    x2_range = task.dataset.x2_range
    x_lbound = np.array([x1_range[0], x2_range[0]])
    x_ubound = np.array([x1_range[1], x2_range[1]])
except:
    x_lbound = task.oracle.internal_dataset.x.min(axis=0)
    x_ubound = task.oracle.internal_dataset.x.max(axis=0)
    y_lbound = task.oracle.internal_dataset.y.min()
    y_ubound = task.oracle.internal_dataset.y.max()
    
print("##################Before Normalization##################")
print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

if normalize_y:
    task.map_normalize_y()
    y_min = task.normalize_y(y_min)[0]
    y_max = task.normalize_y(y_max)[0]
    y_lbound = task.normalize_y(y_lbound)[0]
    y_ubound = task.normalize_y(y_ubound)[0]
if normalize_x:
    task.map_normalize_x()
    x_min = task.normalize_x(x_min)[0]
    x_max = task.normalize_x(x_max)[0]
    x_lbound = task.normalize_x(x_lbound)[0]
    x_ubound = task.normalize_x(x_ubound)[0]

if normalize_x or normalize_y:
    print("##################After Normalization##################")
    print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
    print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
    print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

##################Before Normalization##################
x_min: [ 0.05712451  0.09792493  0.         -0.73992664 -4.1658483  -0.55521834
 -0.8117127   0.30617234  0.21303217  0.7081446  -1.9346311   0.8510519
  0.09275433  0.08746078 -0.11630376  0.0911145   0.         -0.5433565
  2.317303   -0.8797422  -0.71419644  0.31998715  0.20248221  0.6802228
 -1.4124309   0.79242814  0.09484535  0.08976819 -0.11504355 -0.14334697
  0.         -0.49891347  2.5124936  -0.7084233  -0.38319325  0.27876195
  0.24222538  0.60925144 -1.618649    0.761339    0.09221529  0.09010383
  0.0656946  -0.14393781  0.         -0.7393186  -3.6844764  -1.0871363
 -1.2362465   0.3256473  -0.00965876  0.7184322  -1.8095994   0.78017086
  0.09208925  0.08849681] 
 x_max: [ 0.11303348  0.16561559  0.          0.6881478  -2.6678524   0.4853557
  0.25358856  0.47964787  1.4333469   0.8417608  -0.5394638   1.258658
  0.1021037   0.10209594 -0.0742105   0.15999891  0.          0.67675275
  4.279125    0.80026037  1.1559

In [ ]:
import shutil
args = build_args()

seed_everything(args.seed)
torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore", message="The dataloader, .* does not have many workers.*")

try:
    # # 尝试访问 inverse_model
    inverse_model

except NameError:
    inverse_model = None  # 如果未定义，设置为 None

# inverse_model = None
num_epochs=  2000

args.logging_dir = args.logging_dir or f"../logs/run_ours_{args.task}_seed{args.seed}_budget{args.budget}_hidden{args.hidden_size}_epoch{num_epochs}_betamax{0.05}"
print(args.logging_dir )

# logging_dir = args.logging_dir
# while os.path.exists(logging_dir):
#     logging_dir = f"{args.logging_dir}_v{version}"
#     version += 1
# os.makedirs(logging_dir, exist_ok=True)
 
# try:
#     # # 尝试访问 inverse_model
#     inverse_model 

# except NameError:
#     inverse_model = None  # 如果未定义，设置为 None


# inverse_model = mango_train(
#     task=task,
#     task_name=args.task,
#     ckpt_dir=args.logging_dir,
#     hidden_size=args.hidden_size,
#     device=args.device,
#     seed=args.seed,
#     condition_training=False,
#     num_epochs=num_epochs,
#     dropout=0.0,
#     augment=True,
#     lr=5e-5,
#     batch_size=64,
#     debais=True,
# )


In [ ]:

model_ckpt = f"/home/tzhouaq/ManGO_SOO/logs/run_ours_DKittyMorphology-Exact-v0_seed2024_budget2048_hidden2048_epoch2000_betamax0.05/ours-dkitty_morphology/dkitty_morphology-seed2024-hidden2048-score_matchingFalse.ckpt"
inverse_model = DiffusionSOO.load_from_checkpoint(
    model_ckpt,
    taskname=task.dataset_name,
    task=task,
    lr=1e-4,
    dropout_p=0.0,
    hidden_size=args.hidden_size,
    debais=True,
    augment=True,
    condition_training=False
)
# inverse_model.gen_sde.forwardmodel.load_state_dict(model.state_dict())
# inverse_model.forwardmodel.load_state_dict(model.state_dict())


uncondition_training T: {1.0}


In [ ]:
from mango_jupyter_scripts.eval_fks import mango_eval
# This cell is to conduct unconditional generation

augment = True
condition_training = False
 
if augment:

    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

# y_given_list = np.linspace(0.5, 15, 200)
num_samples = 256

import logging

class CustomFilter(logging.Filter):
    def filter(self, record):
        if "is all 0s; this may be normal during initialization" in record.getMessage():
            return False
        return True
logging.getLogger().addFilter(CustomFilter())

res = mango_eval(
    task=task,
    task_name=args.task,
    forwardmodel=forwardmodel,
    inverse_model=inverse_model,
    device='auto',
    logging_dir=args.logging_dir,
    num_samples=num_samples,
    augment=augment,
    clip_dic=clip_dic,
    condition_training=condition_training,
    num_steps=250,
    seed=args.seed,

)

res['x'].shape#, res['y'].shape, res['y_scores'].shape
 

{'simple_clip': True, 'clip_min': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64), 'clip_max': tensor([1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000], dtype=torch.float64)}
the first 5 (no rank) design (normalized_x=True, normalized_y=True) is:
 tensor([[0.5359, 0.1110, 0.0000, 0.6419, 0.7081, 0.5436, 0.6432, 0.6255

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


(256, 56)

In [ ]:

import numpy as np
 
y_unob_max = task.oracle.internal_dataset.y.max()
y_unob_min = task.oracle.internal_dataset.y.min()

designs = res['x']
preds =   res['y_gen']
# preds = res['y']
scores = res['y_scores']
 
print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

  
# task = design_bench.make(args.task)
if task.is_normalized_y:
    best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
else:
    best_training_score = np.max(task.dataset.y)
print("Training dataset:\n The best score during training is: ",best_training_score )
oracle = np.max(scores)
print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
print(f' The corresponding design is: {designs[np.argmax(scores)]}')

k = 1
percentile = 100
one_shot = scores[np.argmax(preds)][0]
normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
 


k = 16
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

k = 128
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')
 

k = 256
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

Input structure:
 The number of the designs is:  56 ; the dimension of the designs is:  56
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [199.36252948]
Oracle results:
 On the given design: the Oracle is: 312.6882019042969
 The corresponding design is: [ 0.08566501  0.10863988  0.          0.16695386 -3.72441     0.07011956
 -0.08144343  0.42428052  0.98327756  0.78132796 -1.4744171   0.92659986
  0.0947741   0.09413385 -0.10001854  0.13414517  0.          0.15581739
  3.800012   -0.07373685  0.1478594   0.511332    0.71551865  0.7371161
 -0.549521    0.89245814  0.09711662  0.09733858 -0.10564853 -0.11436668
  0.         -0.23942372  3.3997962   0.26809925  0.16979277  0.31538492
  0.4983738   0.813089   -1.1779999   0.9241477   0.09479056  0.09832469
  0.08864684 -0.11800491  0.          0.12677658 -3.0552485  -0.7675494
  0.14685738  0.34769398  0.80860585  0.76828194 -1.071461

In [ ]:
augment = True
if normalize_y:
    print('y_given before normalization:',  task.oracle.internal_dataset.y.max())
    y_given = torch.tensor(task.normalize_y(task.oracle.internal_dataset.y.max())[0])
    # y_given = torch.tensor(task.oracle.external_dataset.y.max()) 
else:
    y_given = torch.tensor(task.oracle.internal_dataset.y.max())
print('y_given:', y_given)
if y_max.item() < y_given:
    y_max = y_given 
if  y_min.item() > y_given:
    y_min = y_given

if augment:

    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

args.budget = 256
seed_list = [ 39,40, 41,42,   44, 45, 46, 47, 49] 

reported_results_1shot = []
reported_results_16shot = []
reported_results_128shot = []
reported_results_256shot = []

args.budget = 256
num_steps = 200
inference_scaling_bool = False
duplicated_time = 64
guidance_scals2= 1
description = 'nonresampling'

# y_given = torch.tensor([2])
for seed in seed_list:
    print(f'***********************seed{seed}*************************')
    print(f"Running ours method for {args.task} with seed {seed}")
    
    res = mango_eval(
        task,
        task_name=args.task,
        forwardmodel=forwardmodel,
        inverse_model=inverse_model,
        logging_dir=args.logging_dir,
        hidden_size=args.hidden_size,
        num_samples=args.budget,
        device=args.device,
        seed=seed,
        clip_dic=clip_dic,
        augment=augment,
        gen_condition=y_given,
        guidance=True,
        num_steps=num_steps,
        inference_scaling_bool=inference_scaling_bool,
        duplicated_time=duplicated_time,
        guidance_scals2=guidance_scals2,
        description=description,
    )
 
 
    y_unob_max = task.oracle.internal_dataset.y.max()
    y_unob_min = task.oracle.internal_dataset.y.min()

    designs = res['x']
    preds =   res['y_gen']
    # preds = res['y']
    scores = res['y_scores']

    # designs = designs.reshape(1, -1, designs.shape[-1]) 
    # preds = preds.reshape(1, -1, preds.shape[-1])
    # scores = scores.reshape(1, -1, scores.shape[-1])
    print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
    print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

    
    # task = design_bench.make(args.task)
    if task.is_normalized_y:
        best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
    else:
        best_training_score = np.max(task.dataset.y)
    print("Training dataset:\n The best score during training is: ",best_training_score )
    oracle = np.max(scores)
    print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
    print(f' The corresponding design is: {designs[np.argmax(scores)]}')

    k = 1
    percentile = 100
    one_shot = scores[np.argmax(preds)][0]
    normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - normalize_one_shot))
    reported_results_1shot.append(normalize_one_shot)


    k = 16
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 128
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_128shot.append(normalize_few_shot)

    k = 256
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - normalize_few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_256shot.append(normalize_few_shot)
    
print(f'***********************OVERALL RESULTS*************************')
print(f"reported_results_1shot: {np.mean(reported_results_1shot):.4f} ± {np.std(reported_results_1shot):.4f}")
print(f"reported_results_16shot: {np.mean(reported_results_16shot):.4f} ± {np.std(reported_results_16shot):.4f}")
print(f"reported_results_128shot: {np.mean(reported_results_128shot):.4f} ± {np.std(reported_results_128shot):.4f}")
print(f"reported_results_256shot: {np.mean(reported_results_256shot):.4f} ± {np.std(reported_results_256shot):.4f}")
 



y_given before normalization: 340.90985
y_given: tensor([1.0000], dtype=torch.float64)
{'simple_clip': True, 'clip_min': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64), 'clip_max': tensor([1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000], dtype=torch.float64)}
***********************seed39*************************
Ru

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  56 ; the dimension of the designs is:  56
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [199.36252948]
Oracle results:
 On the given design: the Oracle is: 312.2864990234375
 The corresponding design is: [ 0.08469462  0.10919915  0.          0.14757532 -3.8180523   0.06782663
 -0.0631544   0.42659175  0.97228706  0.7783304  -1.4602828   0.92158836
  0.09423466  0.09408642 -0.09944879  0.13443652  0.          0.1623686
  3.809768   -0.01239681  0.15875661  0.5124271   0.69634175  0.73710203
 -0.59510887  0.8973372   0.09725466  0.09718882 -0.10551343 -0.11255516
  0.         -0.24312294  3.4142044   0.2929929   0.18167281  0.3159095
  0.47616825  0.81522226 -1.1883018   0.93107086  0.09508833  0.09810634
  0.08849211 -0.11522395  0.          0.12479043 -3.054203   -0.8006016
  0.15829039  0.34956136  0.77432877  0.7689039  -1.0759304